### Guardar gestos mano

In [20]:
import cv2
import os
import mediapipe as mp

# --- CONFIGURACIÓN ---
NOMBRE_DATASET = "Dataset_Gestos"
ETIQUETAS = {
    0: "0_Neutro",
    1: "1_Puno",
    2: "2_Palma",
    3: "3_Victoria"
}

# Crear estructura de carpetas si no existen
if not os.path.exists(NOMBRE_DATASET):
    os.makedirs(NOMBRE_DATASET)

for key, nombre_carpeta in ETIQUETAS.items():
    path = os.path.join(NOMBRE_DATASET, nombre_carpeta)
    if not os.path.exists(path):
        os.makedirs(path)

# Configuración de MediaPipe (Solo para visualización)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

# Contadores para nombrar las fotos y no sobreescribir
contadores = {}
for key in ETIQUETAS:
    # Busca cuántas fotos hay ya para seguir la numeración
    path = os.path.join(NOMBRE_DATASET, ETIQUETAS[key])
    contadores[key] = len(os.listdir(path))

print("--- RECOLECTOR DE FOTOS ---")
print(f"Las fotos se guardarán en la carpeta: {NOMBRE_DATASET}")

while True:
    ret, frame = cap.read()
    frame=cv2.flip(frame, 1)  
    if not ret: break
    
    # Hacemos una copia para dibujar encima, PERO guardaremos la original (limpia)
    frame_display = frame.copy()
    
    # Procesar con MediaPipe solo para feedback visual
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    mano_detectada = False
    if results.multi_hand_landmarks:
        mano_detectada = True
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame_display, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Info en pantalla
    fuente = cv2.FONT_HERSHEY_SIMPLEX
    escala = 0.5
    grosor = 1

    texto_estado = f"Puno: {contadores[1]} | Palma: {contadores[2]} | Victoria: {contadores[3]}"
    cv2.putText(frame_display, texto_estado, (10, 30), fuente, escala, (0, 255, 255), 2)

        # === TEXTO INFERIOR EN 2 LÍNEAS ===
    linea1 = "Presiona para guardar -> 0: Neutro | 1: Puno"
    linea2 = "2: Palma | 3: Victoria | q: Salir"

    org1 = (10, 440)
    org2 = (10, 470)

    # Calcular tamaños
    (t1_ancho, t1_alto), bl1 = cv2.getTextSize(linea1, fuente, escala, grosor)
    (t2_ancho, t2_alto), bl2 = cv2.getTextSize(linea2, fuente, escala, grosor)

    # Ancho máximo de ambas líneas
    ancho_total = max(t1_ancho, t2_ancho)

    # Dibujar rectángulo de fondo para ambas líneas
    cv2.rectangle(frame_display,
                (org1[0] - 5, org1[1] - t1_alto - 5),
                (org1[0] + ancho_total + 5, org2[1] + bl2 + 5),
                (0, 0, 0),
                -1)

    # Dibujar ambas líneas
    cv2.putText(frame_display, linea1, org1, fuente, escala, (0, 255, 255), grosor)
    cv2.putText(frame_display, linea2, org2, fuente, escala, (0, 255, 255), grosor)

    if not mano_detectada:
        cv2.putText(frame_display, "MANO NO DETECTADA", (10, 400), fuente, 0.7, (0, 0, 255), 2)

    cv2.imshow('Recolector de gestos de manos', frame_display)
    
    # --- CONTROL DE TECLAS ---
    k = cv2.waitKey(1)
    guardar_clase = -1
    
    if k == ord('0'): guardar_clase = 0
    elif k == ord('1'): guardar_clase = 1
    elif k == ord('2'): guardar_clase = 2
    elif k == ord('3'): guardar_clase = 3
    elif k == ord('q'): break
    
    # Lógica de guardado
    if guardar_clase != -1:
        # Generar nombre de archivo único
        carpeta = ETIQUETAS[guardar_clase]
        contador = contadores[guardar_clase]
        nombre_foto = f"{carpeta}_{contador}.jpg"
        ruta_foto = os.path.join(NOMBRE_DATASET, carpeta, nombre_foto)
        
        cv2.imwrite(ruta_foto, frame)
        print(f"Foto guardada: {ruta_foto}")
        contadores[guardar_clase] += 1

cap.release()
cv2.destroyAllWindows()

--- RECOLECTOR DE FOTOS ---
Las fotos se guardarán en la carpeta: Dataset_Gestos


### Guardar gestos cara

In [ ]:
import cv2
import os
import mediapipe as mp

# --- CONFIGURACIÓN ---
NOMBRE_DATASET = "Dataset_Cara"
ETIQUETAS = {
    0: "0_Neutro",
    1: "1_Boca_Abierta",
    2: "2_Ojos_Cerrados",
    3: "3_Sonrisa"
}

# Crear estructura de carpetas si no existen
if not os.path.exists(NOMBRE_DATASET):
    os.makedirs(NOMBRE_DATASET)

for key, nombre_carpeta in ETIQUETAS.items():
    path = os.path.join(NOMBRE_DATASET, nombre_carpeta)
    if not os.path.exists(path):
        os.makedirs(path)

# --- CAMBIO IMPORTANTE: USAMOS FACE MESH ---
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# refine_landmarks=True añade iris (ojos) para mayor precisión en la mirada
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

cap = cv2.VideoCapture(0)

# Contadores
contadores = {}
for key in ETIQUETAS:
    path = os.path.join(NOMBRE_DATASET, ETIQUETAS[key])
    contadores[key] = len(os.listdir(path))

print("--- RECOLECTOR FACIAL ---")
print(f"Las fotos se guardarán en la carpeta: {NOMBRE_DATASET}")

while True:
    ret, frame = cap.read()
    frame=cv2.flip(frame, 1)  
    if not ret: break
    
    frame_display = frame.copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Procesar cara
    results = face_mesh.process(frame_rgb)
    
    cara_detectada = False
    if results.multi_face_landmarks:
        cara_detectada = True
        for face_landmarks in results.multi_face_landmarks:
            # Dibujamos la malla facial (tesselation)
            mp_drawing.draw_landmarks(
                image=frame_display,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            
            # Dibujamos contornos (ojos, labios) más marcados
            mp_drawing.draw_landmarks(
                image=frame_display,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())

    # Info en pantalla
    fuente = cv2.FONT_HERSHEY_SIMPLEX
    escala = 0.5
    grosor = 1

    texto_estado = f"Neutro: {contadores[0]} | Boca aierta: {contadores[1]} | Ojos cerrados: {contadores[2]} | Sonrisa: {contadores[3]}"
    cv2.putText(frame_display, texto_estado, (10, 30), fuente, escala, (0, 255, 255), 2)

    # === TEXTO INFERIOR EN 2 LÍNEAS ===
    linea1 = "Presiona para guardar -> 0: Neutro | 1: Boca abierta"
    linea2 = "2: Ojos cerrados | 3: Sonrisa | q: Salir"

    org1 = (10, 440)
    org2 = (10, 470)

    # Calcular tamaños
    (t1_ancho, t1_alto), bl1 = cv2.getTextSize(linea1, fuente, escala, grosor)
    (t2_ancho, t2_alto), bl2 = cv2.getTextSize(linea2, fuente, escala, grosor)

    # Ancho máximo de ambas líneas
    ancho_total = max(t1_ancho, t2_ancho)

    # Dibujar rectángulo de fondo para ambas líneas
    cv2.rectangle(frame_display,
                (org1[0] - 5, org1[1] - t1_alto - 5),
                (org1[0] + ancho_total + 5, org2[1] + bl2 + 5),
                (0, 0, 0),
                -1)

    # Dibujar ambas líneas
    cv2.putText(frame_display, linea1, org1, fuente, escala, (0, 255, 255), grosor)
    cv2.putText(frame_display, linea2, org2, fuente, escala, (0, 255, 255), grosor)

    if not cara_detectada:
        cv2.putText(frame_display, "CARA NO DETECTADA", (10, 400), fuente, 0.7, (0, 0, 255), 2)

    cv2.imshow('Recolector Facial', frame_display)
    
    # --- CONTROL DE TECLAS ---
    k = cv2.waitKey(1)
    guardar_clase = -1
    
    if k == ord('0'): guardar_clase = 0
    elif k == ord('1'): guardar_clase = 1
    elif k == ord('2'): guardar_clase = 2
    elif k == ord('3'): guardar_clase = 3
    elif k == ord('q'): break

    # Lógica de guardado
    if guardar_clase != -1:
        # Generar nombre de archivo único
        carpeta = ETIQUETAS[guardar_clase]
        contador = contadores[guardar_clase]
        nombre_foto = f"{carpeta}_{contador}.jpg"
        ruta_foto = os.path.join(NOMBRE_DATASET, carpeta, nombre_foto)
        
        cv2.imwrite(ruta_foto, frame) 
        print(f"Foto guardada: {ruta_foto}")
        contadores[guardar_clase] += 1

cap.release()
cv2.destroyAllWindows()

--- RECOLECTOR FACIAL ---
Las fotos se guardarán en la carpeta: Dataset_Cara
